# **Bitcoin minute-by-minute predictions**

# This is a work in progress. I am updating this code, and it  will be featured in a Medium article. 

## If this notebook gets 10 upvotes I will release the Inference code notebook for real time deployment.

# Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import joblib


# fix random seed for reproducibility
np.random.seed(7)
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

# Load in Data

In [ ]:
path_name = "../input/392-crypto-currency-pairs-at-minute-resolution/btcusd.csv" 
df = pd.read_csv(path_name, index_col='time')
df.index = pd.to_datetime(df.index, unit='ms')
df = df[~df.index.duplicated(keep='first')]
df = df.resample('1T').pad()

In [ ]:
#Get scaled differences between open and close
df['delta'] = (df.close - df.open) / df.open
# Adding a previous close column
df['prev_close'] = df['close'].shift(1)

In [ ]:
df.sample(5)

In [ ]:
df.tail(5)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df.index[-100000:], df.close[-100000:])
plt.show()

# Univariate LSTM

## Prepare Dataset

In [ ]:
def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
dataset = df['close'][-100000:].values
dataset = dataset.astype('float32')
dataset = dataset.reshape(-1, 1)
dataset

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# reshape into X=t and Y=t+1
look_back = 6
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

## Training

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(8))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=2)

## Evaluation

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.figure(figsize=(15,10))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

### Save Model

In [ ]:
# Scaler
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 
# Model
model.save('keras_lstm_uni.h5')

# Thanks for making it to the end! I'm only two bronze medals away from Kaggle Expert (searching for jobs so maybe it would look cool on my LinkedIn?) so please leave a upvote if you enjoyed this notebook and/or know the struggle of job searching!